In [14]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import re
import ollama
from ollama import chat
from ollama import ChatResponse
from duckduckgo_search import DDGS
from transformers import AutoTokenizer 
import nltk
from nltk.corpus import stopwords
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate

from openai import OpenAI

In [15]:
sbert_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-electra-base")

In [16]:
# Charger les données JSON
json_file_path = "linked_files.json"
with open(json_file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)

In [17]:
def extract_technique_section(content):
    # Rechercher la section Technique jusqu'au prochain titre de niveau 1
    match = re.search(r"# Technique\s*(.*?)(?=\n# |\Z)", content, re.DOTALL)
    if match:
        # Extraire le contenu
        technique_text = match.group(1).strip()
        # Supprimer toutes les lignes commençant par un '#' (titres et sous-titres)
        cleaned_text = re.sub(r"^#.*", "", technique_text, flags=re.MULTILINE)
        # Nettoyer les espaces et les lignes vides
        return "\n".join(line.strip() for line in cleaned_text.splitlines() if line.strip())
    return ""

In [18]:
# Extraire les titres et le contenu technique
documents = []
for entry in json_data:
    title = entry["title"]
    content = entry["content"]
    # Extraire uniquement la partie "Technique" du contenu
    technique_section = extract_technique_section(content)
    # Combiner le titre et le contenu technique
    combined_text = f"{title} {technique_section}"
    #print(combined_text)
    documents.append(combined_text)

print(documents)

['Fonctionnement du moteur à pleine charge ', "Production d'air décentralisée Un compresseur d'air placé près de la consommation permet d'éviter l'utilisation d'un réseau d'air entrainant des pertes de charge coûteuses. Pour vérifier la pertinence de cette solution, relevez le fonctionnement des appareils et évaluez les possibilités d'ajouter un compresseur près des zones de consommation.\nLa solution s'applique aux sites industriels ayant un réseau d'air comprimé étendu. par exemple les sites avec plusieurs bâtiments.\nIl n'est peut être pas pertinent de tirer le réseau d'air sur une grande longeur si le bâtiment est peu consomamteur. Dans ce cas, un petit compresseur décentralisé peut être pertinent.\nLa solution est applicable aussi dans le cas où le compresseur alimente une zone indépendante (a pression différente par exemple)", 'Moteur à haut rendement IE4 (super premium) ', 'Evaluer la durée de ventilation d’un palier ', "Programmateur d'intermittence sur une chaudière existante 

In [19]:
# Calculer les embeddings des documents
embeddings = sbert_model.encode(documents)

# Indexer avec FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
faiss.normalize_L2(embeddings)
index.add(embeddings)

In [ ]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="",
)

In [21]:
# Gérer le contexte conversationnel
conversation_history = []  # Stockage de l'historique
history_limit = 6  # Nombre de tours de conversation gardés

def generate_rephrasing(query, contexte_recent):
    """Génère une réponse via Gemma:2b en utilisant un pipeline RAG."""
    
    response = client.chat.completions.create(model="deepseek/deepseek-r1:free", messages=[
        {"role": "system", "content": "Étant donné un historique de conversation et la dernière question de l'utilisateur qui pourrait faire référence à un contexte dans l'historique de conversation, formule une question autonome qui peut être comprise sans avoir besoin de l'historique. Ne réponds PAS à la question, reformulez-la simplement si nécessaire, sinon retourne-la telle quelle. Parle en français"},

        {"role": "user", "content": f"Question : {query}\nContexte :\n{contexte_recent}\nRéponse :"}
    ])

    return response.choices[0].message.content


In [22]:
def recuperer_historique_recent(user_query):
    """Ajoute le contexte conversationnel à la requête."""
    context = " ".join(conversation_history[-history_limit:])
    return f"{context} {user_query}" if context else user_query

# Fonction de recherche
def search(query, top_k=5):
    # Vectoriser la requête 
    query_embedding = sbert_model.encode([query])
    faiss.normalize_L2(query_embedding)
    
    # Rechercher les documents les plus proches avec FAISS
    _, faiss_results = index.search(query_embedding, top_k)
    faiss_scores = [(documents[i], json_data[i]) for i in faiss_results[0]]
    
    # Réordonner les résultats avec le Cross-Encoder
    cross_scores = cross_encoder.predict([(query, doc[0]) for doc in faiss_scores])
    ranked_results = [x for _, x in sorted(zip(cross_scores, faiss_scores), key=lambda pair: pair[0], reverse=True)]    
    return ranked_results[:top_k]

# Fonction pour interroger le RAG
def ask_rag(user_query):
    """Gère la conversation et interroge le RAG."""
    # Ajouter la nouvelle question à l'historique
    conversation_history.append(f"[Utilisateur]: {user_query}")
    
    # Limiter la taille de l'historique
    if len(conversation_history) > history_limit:
        conversation_history.pop(0)

    prompt = generate_rephrasing(user_query, conversation_history)
    # Rechercher des résultats pertinents
    results = search(prompt)  # Assurez-vous que `search` accepte un prompt formaté
    return results


In [23]:
stop_words = set(stopwords.words('french'))

# Fonction pour enlever les mots inutiles (stopwords)
def filter_stop_words(text):
    """Enlève les mots inutiles de la question."""
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

def generate_answer(query, retrieved_info, contexte_recent):
    """Génère une réponse en utilisant un pipeline RAG."""

    prompt = generate_rephrasing(query, conversation_history)

    
    response = client.chat.completions.create(model="deepseek/deepseek-r1:free", messages=[
        {"role": "system", "content": f"""
        Tu es un assistant intelligent. Voici le contexte récent de la conversation : 
        {contexte_recent}

        Voici les documents pertinents sur lesquels tu dois te baser : 
        {retrieved_info}

        L'utilisateur demande : "{query}"

        Les instructions sont :
        - Réponds en français.
        - Réponds à la QUESTION en utilisant exclusivement les DOCUMENTS fournis et en tenant compte du CONTEXTE.
        - Ta réponse doit être concise, claire et formulée avec tes propres mots.
        - Donne des exemples concrets si possible.
        - Fournis des explications détaillées si nécessaire.
        - N'hésite pas à donner plusieurs informations.
        - Si la réponse n'est pas contenue dans les documents, réponds simplement que tu n'as pas trouvé d'information sur ce sujet.
        """},
        {"role": "user", "content": f"Question : {query}\nInfos trouvées :\n{retrieved_info}\nRéponse :"}
    ],
    stream=False
    )

    return response.choices[0].message.content


# Système RAG : Recherche + Génération
def rag_system(question):
    """Système RAG qui combine recherche web et génération de réponse."""
    contexte_recent = recuperer_historique_recent(question)
    
    # Recherche Web pour obtenir des informations pertinentes
    retrieved_info = ask_rag(question)

    # Génération de la réponse via Gemma:2b
    response = generate_answer(question, retrieved_info, contexte_recent)

    conversation_history.append(f"[Système]: {response}")

    if len(conversation_history) > history_limit:
        conversation_history.pop(0)

    return response

In [26]:
# Exemple d'utilisation
user_input = "Comment isoler une maison ?"
response = rag_system(user_input)
print(response)

**Réponse :**

Pour isoler une maison, les documents mentionnent principalement l'**amélioration des fenêtres** via des **fermetures isolantes**. Voici les solutions concrètes :

1. **Vitrages performants** :  
   - Utilisez des fenêtres à **isolation R-3** ou en **vitrage haute performance** (double/triple vitrage).  
   - Coût : entre **50 et 150 €** par fenêtre (exemple : études de cas en France et aux États-Unis).  
   - Gains : **Confort thermique** (été/hiver) et **isolation acoustique**, avec un temps de retour sur investissement (TRI) souvent supérieur à **10 ans**.

2. **Zones stratégiques** :  
   - Privilégiez la façade nord (exemple : étude de cas 626) pour maximiser l’efficacité.

**Limites** : Les documents ne couvrent pas d’autres méthodes (isolation des murs, toiture, etc.). Pour une approche globale, consultez des sources complémentaires.


In [30]:
# Exemple d'utilisation
user_input = "Comment je peux le faire différemment"
response = rag_system(user_input)
print(response)